In [1]:
import numpy as np
import time
from pydgrid.pydgrid import grid
from pydgrid.pf import pf_eval,time_serie
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import push_notebook
from bokeh.resources import INLINE
output_notebook(INLINE)

Loading BokehJS ...

## 907 bus 3 wire system with transformer

### Execute power flow 

In [2]:
sys1 = grid()
sys1.read('n1_f1.json')  # Load data
sys1.read_loads_shapes('n1_f1_load_shapes.json')  
sys1.pf_solver = 2
sys1.pf()  # solve power flow
        
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

### Graph with obtained results

In [3]:
sys1.s_radio_scale =0.5
sys1.s_radio_min =5
sys1.s_radio_max =100
sys1.snapshot(60*60*12)
sys1.pf() 
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents
sys1.bokeh_tools()

p_grid = figure(width=900, height=800,
           title='3 bus 4 wire system with transformer') 

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p_grid.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p_grid.circle(source=source, x='x', y='y', size='s_radio', color="s_color", alpha=0.5)

p_grid.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p_grid.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update_grid(t_h=0.0):
    sys1.snapshot(60*60*t_h)

    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()
    


#p_grid = gridplot([[p], [p_2]])
show(p_grid, notebook_handle=True)



In [4]:
from ipywidgets import interact
interact(update_grid, t_h=(0,24, 0.01), continuous_update=False)

A Jupyter Widget

<function __main__.update_grid>

### Interaction 

In [5]:
p = figure(width=600, height=300,
           title='Voltage vs load powers', 
           y_range = [0.95,1.01], #x_range = [50,-300], 
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)
#cr = p1.circle(source=source, x='y', y='v_an_pu', size=15, color="red", alpha=0.5)
p.circle(source=source, x='x', y='v_an_pu', size=15, color="red", alpha=0.5)
p.circle(source=source, x='x', y='v_bn_pu', size=15, color="green", alpha=0.5)
p.circle(source=source, x='x', y='v_cn_pu', size=15, color="blue", alpha=0.5)

#p1.circle(source=source, x='y', y='v_bn_pu', size=15, color="green", alpha=0.5)
#p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
#p.line([-300, 50],[231*1.05,231*1.05], color='red', line_width=5)
#p.line([-300, 50],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))

def update(t_h=0.0):
    sys1.snapshot(60*60*t_h)

    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


In [6]:
from ipywidgets import interact
interact(update, t_h=(0,24, 0.01))

A Jupyter Widget

<function __main__.update>